# BTC Prediction With LSTM

In [24]:
import csv
import numpy as np
from datetime import date, datetime, timedelta
from lstm import Machine
import pandas as pd
import tensorflow as tf
from plotly.offline import init_notebook_mode
import cufflinks as cf

MODEL_PATH = "data/lstm.hd5"
CSV_PATH = "data/binance-BTCUSD.csv"
TRAIN_TEST_RATE = 0.85
NUM_SEQ = 30
DUR_PRED = 15

cf.go_offline()
init_notebook_mode(connected=True)

with open(CSV_PATH) as fin:
    reader = [dict(item) for item in csv.DictReader(fin)]
machine = Machine(reader)
(_, _, X_test, Y_test) = machine.split_data(
    TRAIN_TEST_RATE, NUM_SEQ, DUR_PRED
)

base_date = datetime.utcfromtimestamp(
    int(reader[-1]['openTime']) / 1000
).date() - timedelta(days=len(Y_test) + 1)
max_date = datetime.utcfromtimestamp(
    int(reader[-1]['openTime']) / 1000
).date() - timedelta(days=30)

user_date = None

while not user_date:
    try:
        user_date = date.fromisoformat(input(
            "What date you want to see? ("
            f"min: {base_date.isoformat()} "
            f"max: {max_date.isoformat()}) "
        ))
#         if not (base_date <= user_date <= max_date):
#             user_date = None
    except ValueError:
        continue

df_index = (user_date - base_date).days
X_test = X_test[df_index, :]
X_test = X_test.reshape((1, X_test.shape[0], X_test.shape[1]))
Y_test = Y_test[df_index, :]
Y_test = Y_test.reshape((1, Y_test.shape[0], Y_test.shape[1]))

from IPython.display import display
display(X_test)

model = tf.keras.models.load_model(MODEL_PATH)
pred = model.predict(X_test)

pred = np.array([
    machine.scaler.inverse_transform(row)
    for row in pred
])
Y_test = np.array([
    machine.scaler.inverse_transform(row)
    for row in Y_test
])

from IPython.display import display
display(pred, Y_test)

# Needs to change

# for (c, key) in enumerate(['open', 'close', 'high', 'low']):
#     pred_df = pd.DataFrame(
#         {f'Pred': [
#             item[c] for item in pred[df_index]
#         ]},
#         index=pd.date_range(
#             user_date,
#             user_date + timedelta(days=len(pred[df_index]) - 1)
#         )
#     )
#     act_df = pd.DataFrame(
#         {f'Act': [
#             item[c] for item in Y_test[df_index]
#         ]},
#         index=pd.date_range(
#             user_date,
#             user_date + timedelta(days=len(Y_test[df_index]) - 1)
#         )
#     )
#     pd.concat([pred_df, act_df], axis=1).iplot(
#         asFigure=False,
#         kind='scatter',
#         xTitle='Date',
#         yTitle='Price',
#         title=f'Price Prediction ({key})'
#     )

What date you want to see? (min: 2018-04-21 max: 2019-03-21) 2019-04-18


array([[[0.04976343, 0.05014001, 0.04427382, 0.07336073],
        [0.05020531, 0.05133898, 0.0449293 , 0.07398267],
        [0.05134263, 0.05046991, 0.04415398, 0.0721978 ],
        [0.05042248, 0.04694715, 0.04374725, 0.06829223],
        [0.04691511, 0.04772824, 0.0413069 , 0.06862933],
        [0.0477499 , 0.05335234, 0.04669481, 0.07131524],
        [0.05344968, 0.05270887, 0.04619245, 0.07551137],
        [0.05273666, 0.05744946, 0.05127713, 0.07691519],
        [0.05750814, 0.05768322, 0.05226308, 0.07869751],
        [0.05768074, 0.05749345, 0.05081775, 0.08064551],
        [0.05739516, 0.06004534, 0.05339489, 0.07965335],
        [0.06003761, 0.10483271, 0.0981402 , 0.08433951],
        [0.10476783, 0.10956513, 0.12095922, 0.12339897],
        [0.10950039, 0.10743237, 0.10667963, 0.12489645],
        [0.10736949, 0.11411154, 0.10602233, 0.13149939]]])

array([[[4115.972 , 4124.7734, 4206.284 , 4004.9438],
        [4184.043 , 4238.85  , 4305.487 , 4131.9204],
        [4208.0396, 4246.3447, 4324.818 , 4103.848 ],
        [4145.956 , 4190.5576, 4264.9414, 4073.9648],
        [4219.1343, 4263.4023, 4340.4146, 4152.1553],
        [4358.3115, 4399.4062, 4479.776 , 4280.416 ],
        [4383.7495, 4427.9487, 4511.16  , 4300.762 ],
        [4484.2715, 4531.099 , 4619.1885, 4394.1934],
        [4513.193 , 4560.7915, 4649.585 , 4421.4346],
        [4540.577 , 4582.853 , 4674.7554, 4445.459 ],
        [4590.537 , 4627.3965, 4725.3403, 4491.6567],
        [5259.4077, 5298.55  , 5423.3037, 5129.5073],
        [5261.093 , 5306.8755, 5437.9814, 5114.8154],
        [5026.1426, 5084.5137, 5199.7383, 4899.3013],
        [4942.4673, 4998.9155, 5113.8193, 4826.6216]]], dtype=float32)

array([[[5004.96, 5043.89, 5205.  , 4928.59],
        [5042.07, 5170.27, 5234.  , 5026.  ],
        [5170.27, 5236.9 , 5305.  , 5039.  ],
        [5238.38, 5150.  , 5238.4 , 5076.68],
        [5150.  , 5308.25, 5422.  , 5135.  ],
        [5307.86, 5017.37, 5332.67, 4927.  ],
        [5017.37, 5048.01, 5080.58, 4861.22],
        [5047.  , 5045.22, 5099.  , 5004.  ],
        [5047.45, 5131.3 , 5152.99, 5000.  ],
        [5131.28, 5024.95, 5167.38, 4950.  ],
        [5024.95, 5173.72, 5197.72, 5003.94],
        [5173.72, 5202.82, 5230.4 , 5146.8 ],
        [5202.41, 5258.44, 5287.  , 5198.8 ],
        [5258.44, 5258.68, 5320.  , 5175.  ],
        [5258.68, 5291.73, 5333.42, 5230.1 ]]])

# BTC Prediction With GRU

In [1]:
import csv
import numpy as np
from datetime import datetime, timedelta
from gru import Machine
import pandas as pd
import tensorflow as tf
from plotly.offline import init_notebook_mode
import cufflinks as cf

MODEL_PATH = "data/gru.hd5"
CSV_PATH = "data/binance-BTCUSD.csv"
TRAIN_TEST_RATE = 0.85

cf.go_offline()
init_notebook_mode(connected=True)

with open(CSV_PATH) as fin:
    reader = [dict(item) for item in csv.DictReader(fin)]
machine = Machine(reader)
(_, _, X_test, Y_test) = machine.split_data(TRAIN_TEST_RATE)

times = [int(item['openTime']) for item in reader]
times = times[int(len(times) * TRAIN_TEST_RATE) + 4:]
times = [datetime.utcfromtimestamp(t / 1000.0) for t in times]

model = tf.keras.models.load_model(MODEL_PATH)
pred = model.predict(X_test)

pred = machine.scaler.inverse_transform(pred)
Y_test = np.delete(Y_test, len(Y_test) - 1, 0)
Y_test = machine.scaler.inverse_transform(Y_test)

for (c, key) in enumerate(['open', 'close', 'high', 'low']):
    pred_df = pd.DataFrame({
        'Predictions': [item[c] for item in pred],
    }, index=times)
    act_df = pd.DataFrame({
        'Actual': [item[c] for item in Y_test]
    }, index=times[:-1])
    pd.concat([pred_df, act_df], axis=1).iplot(
        asFigure=False,
        kind='scatter',
        xTitle='Date',
        yTitle='Price',
        title=f'Price Prediction ({key})'
    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


KeyboardInterrupt: 

In [4]:
a = None
while not a:
    try:
        a = int(input("test"))
    except ValueError:
        continue
print(a)

testa
testa
test10
10
